<br><br><br>
<b><font size=7>Vehicle Reviews:</font></b><br><br><br>
<font size=6>&emsp;&ensp;Business insight from text mining and word frequency calculation</font><br><br>
***
<br>

- Import libraries

In [1]:
import pandas as pd
import requests
import itertools
import string
import nltk
import spacy
import seaborn as sns

from bs4 import BeautifulSoup
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

# Show all columns
pd.set_option('display.max_columns', None)

<br><br>
<font size=7><b>A</b></font>&emsp;<font size=4>Develop two scrape scripts. The first to fetch the names of major car
companies from the wikipedia page ‘List of automobile manufacturers’. The second to collect ~5000 or more messages posted in Edmunds.com discussion forums on cars. Present the results of both scrapers in a clean .csv file. The first
file should have a list of manufacturers and the second file should have a three columns: date, userid and message.</font>

- <b>Names of major car companies from the wikipedia page ‘List of automobile manufacturers’.</b>

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_automobile_manufacturers"
headers = {'User-Agent': 'Mozilla/5.0'}

page = requests.get(url, headers=headers)
autoWiki = BeautifulSoup(page.text, 'html.parser')

In [3]:
auto = []
country = autoWiki.find_all("div", class_ = "div-col")

for i in range(len(country)):
    for brand in country[i].find_all("li"):
        auto.append(brand.text)

auto = pd.DataFrame({"brand": auto})

In [4]:
print("Found",len(auto),"automobile manufacturers.")
auto.to_csv("auto.csv", index = False)

Found 883 automobile manufacturers.


- <b>Messages posted in Edmunds.com discussion forums on cars.</b><br>
<a href="https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans" style="color:blue">Entry Level Luxury Performance Sedans</a><br>
<a href="https://forums.edmunds.com/discussion/2444/general/x/what-car-is-right-for-me-help-me-choose" style="color:blue">What Car is Right For Me? Help Me Choose!</a><br>
<a href="https://forums.edmunds.com/discussion/8/general/x/car-commercials-the-good-the-bad-and-the-annoying" style="color:blue">Car Commercials, the good, the bad, and the annoying!</a>

In [5]:
date = []
userid = []
message = []

headers = {'User-Agent': 'Mozilla/5.0'}
urls = ["https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p",
       "https://forums.edmunds.com/discussion/2444/general/x/what-car-is-right-for-me-help-me-choose/p",
       "https://forums.edmunds.com/discussion/8/general/x/car-commercials-the-good-the-bad-and-the-annoying/p"]

# Go thru different general discussions
for u in urls:
    # Go thru first 50 pages in each discussion
    for page in range(1,51):
        url = u+str(page)
        page = requests.get(url, headers=headers)
        discussion = BeautifulSoup(page.text, 'html.parser')
        
        # Go thru all comments in one page (normally 50 comments per page)
        comments = discussion.find_all("div", class_ = "Comment")
        for i in comments:
            date.append(i.find_all("time")[0].attrs["datetime"])
            userid.append(i.find_all("a", class_ = "Username js-userCard")[0].text)
            message.append(i.find_all("div", class_ = "Message userContent")[0].text)

comments = pd.DataFrame({"date": date, "userid": userid, "message": message})

In [6]:
print("Found",len(comments),"comments.")
comments.to_csv("comments.csv", index = False)

Found 7500 comments.


<br><br>
<font size=7><b>B</b></font>&emsp;<font size=4>Use the list of manufacturers to help you find the top 5 brands from the forum messages by calculating frequency counts. Do not to count a mention more than once per post, even if it is mentioned multiple times. But if the same post mentions two or more brands then each of those brands should get their
frequency count increased.</font>

In [7]:
auto = pd.read_csv("auto.csv")
comments = pd.read_csv("comments.csv")

In [8]:
# Function checking if a keywork exists in a string
def existKeyword(brand, message):
    # For brand names with multiple words
    if " " in brand:
        return int(brand.lower() in message.replace("\n","").lower())
    # For brand names with a single word
    else:
        return int(brand.lower() in message.replace("\n","").lower().split())

frequency = []

for brand in auto["brand"]:
    comments[brand] = comments["message"].apply(lambda x: existKeyword(brand, x))
    frequency.append(sum(comments[brand]))

auto["frequency"] = frequency

- Top 5 brands<a name="topAuto"></a>

In [9]:
# Show frequency result (first 5)
display(auto.sort_values(by=["frequency"], ascending = False).head(5))
topAutoList = list(auto.sort_values(by=["frequency"], ascending = False).head(5)["brand"])

,brand,frequency
237,BMW,622
420,Honda,478
433,Toyota,412
413,Acura,357
424,Lexus,314


<br><br>
<font size=7><b>C</b></font>&emsp;<font size=4>Calculate the frequency of co-mentions of brands. For example, if Honda and Toyota are mentioned in the same post then the co-mention frequency of Honda and Toyota increases by 1. Which are the top 3 brand pairs which get mentioned together the most? How did you handle the co-mention frequency if 3 brands are mentioned in the same post?</font>

In [10]:
# Count how many brands are mentioned in a comment
comments["mentioned"] = comments.loc[:, ~comments.columns.isin(["date","userid","message"])].sum(axis = 1)

# List all mentioned brands
comments["mentionedBrand"] = comments.loc[:, ~comments.columns.isin(["date","userid","message","mentioned"])].apply(lambda x: list(x[x == 1].index), axis=1)

# Show the comments mentioning more than 1 brands (first 5)
comments[comments.mentioned > 1][["mentionedBrand"]].head()

,mentionedBrand
0,"[Audi, BMW, MAN, BET, CTS, Acura, Lexus, Toyot..."
1,"[Honda, Lexus]"
12,"[Audi, BMW, Infiniti]"
17,"[Lexus, Toyota]"
20,"[BMW, Infiniti]"


In [11]:
# Use itertools.combinations() to find all co-mentions given a set of brands
comments["co-mentions"] = comments["mentionedBrand"].map(lambda x: list(itertools.combinations(x, 2)))

# Show the result (first 5)
comments[comments.mentioned > 1][["co-mentions"]].head()

,co-mentions
0,"[(Audi, BMW), (Audi, MAN), (Audi, BET), (Audi,..."
1,"[(Honda, Lexus)]"
12,"[(Audi, BMW), (Audi, Infiniti), (BMW, Infiniti)]"
17,"[(Lexus, Toyota)]"
20,"[(BMW, Infiniti)]"


In [12]:
# Get all the co-mentions and store them in co (the same co-mention might appear multiple times)
co = []
for i in comments["co-mentions"].to_list():
    co.extend(i)

# Get the UNIQUE co-mentions (a co-mention appears ONLY ONE time)
coList = list(set(co))

# Count the occurrence of each co-mentions
coCount = []
for i in coList:
    coCount.append(co.count(i))

coMentions = pd.DataFrame({"co-mentions": coList, "occurrence": coCount})

- Top 3 co-mentions<a name="topCo"></a>

In [13]:
# Show the top 3 co-mentions (brand pairs) which get mentioned together the most
coMentions.sort_values(by=["occurrence"], ascending = False).head(3)

,co-mentions,occurrence
943,"(Honda, Toyota)",110
239,"(BMW, Infiniti)",89
941,"(BMW, Acura)",86


<br><br>
<font size=7><b>D</b></font>&emsp;<font size=4>Clean the posts by removing unnecessary words and find the 5 most frequently mentioned attributes of cars in the discussions. Which attributes are most strongly associated with the top 5 brands you found in part A.</font>

- Remove **\\n** and replace **\\'** with **'**, **n't** with **not**

In [14]:
# Remove \n
# Replace \' with '
# Replace n't with not
comments["message"] = comments["message"].apply(lambda x: x.lower().\
                                                replace("\n","").\
                                                replace("\\\'","\'").\
                                                replace("n\'t"," not"))

- Remove **punctuation**

In [15]:
# Remove punctuation
def removePunc(textList):
    result = []
    for text in textList:
        word = ""
        for i in text:
            if i not in string.punctuation:
                word = word + i
        result.append(word)
    return result

# Tokenize the messages then remove the punctuation
comments["token"] = comments["message"].apply(lambda x: removePunc(word_tokenize(x)))

# Squeeze all tokens into one list
allMsg = []
for i in comments["token"].to_list():
    allMsg.extend(i)

# Tokens from all messages WITH stopwords (first 30)
allMsg[:30]

['i',
 'personally',
 'think',
 'that',
 'with',
 'a',
 'few',
 'tweaks',
 'the',
 'c320',
 'could',
 'also',
 'sit',
 'at',
 'the',
 'top',
 'of',
 'this',
 'group',
 '',
 'it',
 's',
 'still',
 'more',
 'of',
 'a',
 'entrylevel',
 'luxury',
 'sedan',
 'than']

- Remove **stopwords**

In [16]:
# Remove stopwords from nltk
stopWords = nltk.corpus.stopwords.words('english')
filteredMsg = [w for w in allMsg if not w.lower() in stopWords]

# Remove stopwords from spacy
stopWords = spacy.load('en_core_web_sm').Defaults.stop_words
filteredMsg = [w for w in allMsg if not w.lower() in stopWords]

# Tokens from all messages WITHOUT stopwords (first 30)
filteredMsg[:30]

['personally',
 'think',
 'tweaks',
 'c320',
 'sit',
 'group',
 '',
 's',
 'entrylevel',
 'luxury',
 'sedan',
 'mercedes',
 'lead',
 'believe',
 '',
 'c320',
 '',
 'sport',
 '',
 'needs',
 's',
 'suspension',
 'retuned',
 'accept',
 '17',
 'inch',
 'tires',
 'harsh',
 'ride',
 '']

- Remove **too common words**

In [17]:
cnt = Counter()
for text in filteredMsg:
    for word in text.split():
        cnt[word] += 1

# Top common words (first 30)
display(cnt.most_common(30))

[('s', 5656),
 ('car', 5590),
 ('like', 2922),
 ('cars', 2440),
 ('new', 1926),
 ('think', 1608),
 ('m', 1584),
 ('good', 1569),
 ('better', 1253),
 ('drive', 1188),
 ('bmw', 1148),
 ('tl', 1139),
 ('want', 1098),
 ('know', 1094),
 ('people', 981),
 ('ve', 966),
 ('accord', 957),
 ('time', 915),
 ('d', 903),
 ('buy', 894),
 ('ad', 894),
 ('driving', 859),
 ('years', 810),
 ('honda', 801),
 ('commercial', 800),
 ('price', 775),
 ('best', 767),
 ('3', 765),
 ('engine', 752),
 ('performance', 749)]

In [18]:
def removeWords(text, toRemove):
    result = []
    for i in text:
        if i not in list(toRemove):
            result.append(i)
    return result

# Remove the top 4 common words, for example: "cars"
filteredMsg = removeWords(filteredMsg, set([w for (w, wc) in cnt.most_common(4)]))

- Remove **rare words**

In [19]:
# Find the rare words that are only mentioned once
rareWords = pd.concat([pd.Series(cnt.most_common()),pd.Series(cnt.most_common()).apply(lambda x: x[1])],axis = 1).rename(columns={0:"pair", 1:"cnt"})
rareWords = rareWords[rareWords["cnt"] == 1]

# Remove the rare words
filteredMsg = removeWords(filteredMsg, set([w for (w, wc) in list(rareWords["pair"])]))

- Get the **pos_tag** and **lemmatize** words based one the tags

In [20]:
# Lemmatize the words

# Get the pos tag of a word
def getPos(word):
    tag = pos_tag([word])[0][1].lower()
    if tag.startswith("n"):
        return wordnet.NOUN
    elif tag.startswith("v"):
        return wordnet.VERB
    elif tag.startswith("j"):
        return wordnet.ADJ
    elif tag.startswith("r"):
        return wordnet.ADV
    else:
        return ""

# Function to lemmatize a word
def lemma(word):
    # if the pos tag is not noun, verb, adjective or adverb, then don't care
    if getPos(word) == "":
        return ""
    else:
        return lemmatizer.lemmatize(word, pos = getPos(word))
    
# Function to lemmatize an ADJECTIVE
def lemmaAdj(word):
    # if the pos tag is not noun, verb, adjective or adverb, then don't care
    if getPos(word) == wordnet.ADJ:
        return lemmatizer.lemmatize(word, pos = getPos(word))
    else:
        return ""
    
lemmatizer = WordNetLemmatizer()
lemmaMsg = pd.Series(filteredMsg).apply(lambda x: lemma(x))
lemmaAdjMsg = pd.Series(filteredMsg).apply(lambda x: lemmaAdj(x))

In [21]:
comparison = pd.DataFrame({"Before": list(filteredMsg), "After": list(lemmaMsg)})
comparison[comparison.Before != comparison.After].head()

,Before,After
2,tweaks,tweak
18,needs,need
21,17,
23,tires,tire
28,needs,need


- **Most frequently mentioned words** *(including nouns, verbs, adjectives and adverbs)*

In [22]:
# Check the frequency (lemmaMsg)

# Method 1: using list.count(keyWord)
frequency = []
wordList = list(set(lemmaMsg))[1:]
for word in wordList:
    frequency.append(list(lemmaMsg).count(word))

wordFreq = pd.DataFrame({"Word": wordList, "Occurrence": frequency})

# Method 2: using Counter()
cnt = Counter()
for text in lemmaMsg:
    for word in text.split():
        cnt[word] += 1

In [23]:
print("Result considering all words:\n")
display(wordFreq.sort_values(by="Occurrence", ascending = False).head(10))
display(cnt.most_common(10))

Result considering all words:



,Word,Occurrence
3811,drive,2222
2231,look,2056
8735,new,1945
4219,think,1843
5541,m,1588
1938,ad,1587
4467,good,1572
4532,year,1450
8725,want,1405
7029,get,1322


[('drive', 2222),
 ('look', 2056),
 ('new', 1945),
 ('think', 1843),
 ('m', 1588),
 ('ad', 1587),
 ('good', 1572),
 ('year', 1450),
 ('want', 1405),
 ('get', 1322)]

- **Most frequently mentioned ADJECTIVES** *(excluding nouns, verbs and adverbs)*

In [24]:
# Check the frequency (lemmaAdjMsg)

# Method 1 (slower): using list.count(keyWord) 
#frequency = []
#wordList = list(set(lemmaMsg))[1:]
#for word in wordList:
#    frequency.append(list(lemmaMsg).count(word))

#wordFreq = pd.DataFrame({"Word": wordList, "Occurrence": frequency})

# Method 2 (faster): using Counter()
cnt = Counter()
for text in lemmaAdjMsg:
    for word in text.split():
        cnt[word] += 1

- Top brands and top attributes<a name="topAttr"></a>

In [25]:
freqWordList = []
count = 0
row = 0
while count < 5:
    if cnt.most_common()[row][0] != "commercial":
        freqWordList.append(cnt.most_common()[row][0])
        count += 1
    row += 1

print("Result considering ONLY ADJECTIVE words:\n")
print("Top 5 auto brands:", topAutoList)
print("Top 10 frequently mentioned words:", freqWordList)

Result considering ONLY ADJECTIVE words:

Top 5 auto brands: ['BMW', 'Honda', 'Toyota', 'Acura', 'Lexus']
Top 10 frequently mentioned words: ['new', 'good', 'great', 'best', 'old']


- Check if each attribute **is mentioned** in top brands' comments

In [26]:
# Select comments the mentioned top brands
topAutoComments = comments.loc[(comments["BMW"] == 1) | \
                               (comments["Honda"] == 1) | \
                               (comments["Toyota"] == 1) | \
                               (comments["Acura"] == 1) | \
                               (comments["Lexus"] == 1)][["message","BMW","Honda","Toyota","Acura","Lexus"]]

for word in freqWordList:
    # Check if top attributes are mentioned
    topAutoComments[word] = topAutoComments["message"].apply(lambda x: int(word in x.lower()))

In [27]:
# Count the occurrence of top attributes
result = {}
for brand in topAutoList:
    result[brand] = list(topAutoComments[topAutoComments[brand] == 1].\
                        loc[:,~topAutoComments.columns.isin(["message","BMW","Honda","Toyota","Acura","Lexus"])].sum())


In [28]:
result = pd.DataFrame(result, index = freqWordList).T
display(result)

,new,good,great,best,old
BMW,164,150,92,107,136
Honda,181,130,76,94,123
Toyota,132,114,41,56,94
Acura,111,85,63,53,66
Lexus,109,56,38,33,66


- Transform mentioned times into **percentages** 
### <font color="blue">$\frac{Times\ of\ \mathbf{attribute\ and\ brand}\ mentioned\ together}{Total\ times\ of\ \mathbf{brand}\ mentioned}$</font><a name="topCoAttr"></a>

In [29]:
# Transform result by percentage
resultPcnt = result.copy()

# Number of messages mentioning a certain brand
totalMentioned = []
for i in resultPcnt.index:
    totalMentioned.append(auto.set_index("brand").loc[i,"frequency"])

# Transform: (keyword mentioned times) / (total mentioned times)
resultPcnt["TotalMentioned"] = totalMentioned
resultPcnt = resultPcnt.iloc[:,:-1].divide(list(resultPcnt["TotalMentioned"]),axis = 0)

def make_pretty(styler):
    styler.background_gradient(cmap = sns.color_palette("vlag", as_cmap=True), axis = None)
    styler.format(lambda x: round(x,3))
    return styler

resultPcnt.style.pipe(make_pretty)

,new,good,great,best,old
BMW,0.264,0.241,0.148,0.172,0.219
Honda,0.379,0.272,0.159,0.197,0.257
Toyota,0.32,0.277,0.1,0.136,0.228
Acura,0.311,0.238,0.176,0.148,0.185
Lexus,0.347,0.178,0.121,0.105,0.21


<br><br>
<font size=7><b>E</b></font>&emsp;<font size=4>Which is the most aspirational brand in your data in terms of people actually wanting to buy or own? Describe your thinking behind your analysis.</font>

- Include more frequently mentioned attributes to gain more insights

In [30]:
cnt = Counter()
for text in lemmaAdjMsg:
    for word in text.split():
        cnt[word] += 1
        
freqWordList = []
count = 0
row = 0

# Extend the frequently mentioned words to top 14
while count < 14:
    if cnt.most_common()[row][0] != "commercial":
        freqWordList.append(cnt.most_common()[row][0])
        count += 1
    row += 1

In [31]:
topAutoComments = comments.loc[(comments["BMW"] == 1) | \
                               (comments["Honda"] == 1) | \
                               (comments["Toyota"] == 1) | \
                               (comments["Acura"] == 1) | \
                               (comments["Lexus"] == 1)][["message","BMW","Honda","Toyota","Acura","Lexus"]]

for word in freqWordList:
    topAutoComments[word] = topAutoComments["message"].apply(lambda x: int(word in x.lower()))

In [32]:
result = {}
for brand in topAutoList:
    result[brand] = list(topAutoComments[topAutoComments[brand] == 1].\
                        loc[:,~topAutoComments.columns.isin(["message","BMW","Honda","Toyota","Acura","Lexus"])].sum())
result = pd.DataFrame(result, index = freqWordList).T
display(result)

,new,good,great,best,old,bad,big,little,nice,high,different,small,manual,low
BMW,164,150,92,107,136,67,79,66,69,94,86,55,46,114
Honda,181,130,76,94,123,46,65,48,47,77,36,38,26,81
Toyota,132,114,41,56,94,37,38,34,39,71,21,20,17,72
Acura,111,85,63,53,66,36,29,27,43,55,42,43,14,62
Lexus,109,56,38,33,66,38,33,23,33,41,32,25,15,45


In [33]:
# Transform result by percentage
resultPcnt = result.copy()

# Number of messages mentioning a certain brand
totalMentioned = []
for i in resultPcnt.index:
    totalMentioned.append(auto.set_index("brand").loc[i,"frequency"])

# Transform: (keyword mentioned times) / (total mentioned times)
resultPcnt["TotalMentioned"] = totalMentioned
resultPcnt = resultPcnt.iloc[:,:-1].divide(list(resultPcnt["TotalMentioned"]),axis = 0)

def make_pretty(styler):
    styler.background_gradient(cmap = sns.color_palette("vlag", as_cmap=True), axis = None)
    styler.format(lambda x: round(x,2))
    return styler

resultPcnt.style.pipe(make_pretty)

,new,good,great,best,old,bad,big,little,nice,high,different,small,manual,low
BMW,0.26,0.24,0.15,0.17,0.22,0.11,0.13,0.11,0.11,0.15,0.14,0.09,0.07,0.18
Honda,0.38,0.27,0.16,0.2,0.26,0.1,0.14,0.1,0.1,0.16,0.08,0.08,0.05,0.17
Toyota,0.32,0.28,0.1,0.14,0.23,0.09,0.09,0.08,0.09,0.17,0.05,0.05,0.04,0.17
Acura,0.31,0.24,0.18,0.15,0.18,0.1,0.08,0.08,0.12,0.15,0.12,0.12,0.04,0.17
Lexus,0.35,0.18,0.12,0.11,0.21,0.12,0.11,0.07,0.11,0.13,0.1,0.08,0.05,0.14


<br>

- **<font color="#6CBB3C">Successful</font>** Capture<a name="sExp"></a>

In [34]:
pd.set_option('display.max_colwidth', None)
pd.DataFrame(topAutoComments.iloc[102,]).T[["message","BMW","best"]]

,message,BMW,best
319,"first, is is not really a performance car, rwd dos not make akar performance. second, we all know that bmw makes one of the best handling cars in the world and it have nothing to do with it's being rwd. if you know mini couper is also made by bmw and it handle better than 3 series, and it's fwd.",1,1


<br>

- **<font color="tomato">Faulty</font>** Capture<a name="fExp"></a>

In [35]:
pd.DataFrame(topAutoComments.iloc[131,]).T[["message","Acura","new","old"]]

,message,Acura,new,old
423,looks like subaru is finally serious about competing with the germans and the other japanese auto makers. notice the new legacy borrows a few styling cues from both the new and old acura tl.,1,1,1


<a name="aspirational"></a>
**Honda** will be the most aspirational brands in my opinion in terms of people <u>actually wanting to buy or own</u> for the following reasons:

1. **38%** of the comments that mentioned **Honda** also mentioned the attributes **new**. That indicates people might be discussing more about the new models from Honda. In that case, I conclude that people will have higher possibility considering purchase vehicles from Honda.
2. **27%** of these comments also mentioned **good** which is the second highest percentage among the top 5 brands.
3. **26%** of these comments mentioned **old**. While I was looking into the comments on the forum, I found when people used the word **old**, it is actually often mentioned in a positive way. For example, "*My old Honda works perfectly still*".

Even though **BMW** also has a good amount of comments mentioning **good** and **old**, it has less percentage about **new**. Thus, **Honda** remains my answer for this question.

***
<br>
<font size=7><b>F</b></font>&emsp;<font size=4>Appendices: Other details, summary and thoughts.</font>

1. **Forum chosen (URL)**<br>
<a href="https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans">Entry Level Luxury Performance Sedans</a><br>
<a href="https://forums.edmunds.com/discussion/2444/general/x/what-car-is-right-for-me-help-me-choose">What Car is Right For Me? Help Me Choose!</a><br>
<a href="https://forums.edmunds.com/discussion/8/general/x/car-commercials-the-good-the-bad-and-the-annoying">Car Commercials, the good, the bad, and the annoying!</a><br><br>
2. **Top 5 brands with the frequency table.**<br>
Please find it [here](#topAuto)<br><br>
3. **Top 3 co-mentioned brand pairs with the frequency table.**<br>
Please find it [here](#topCo)<br><br>
4. **Top 5 most mentioned attributes with a list with frequency of mentions.<br>Another list of co-mention of attributes and the top 5 brands found earlier.**<br>
Please find the 5 most mentioned attributes [here](#topAttr)<br>
Please find the list of co-mention of attributes and the top 5 brands I found earlier [here](#topCoAttr) (*by occurrence/percentage*)<br><br>
5. **A description for how I decided something was aspirational and how you measured and found the brand.**<br>
Please find it [here](#aspirational)<br><br>
7. **A note of all insights that I have gained through my work.**<br>
Bag-of-words, the method I practiced in this work, is a good approach to capture the popularity of some topics (in this case, the car brands).<br>
Yet, we're <u>not able to identify if some words are mentioned together in a positive or negative way</u>. Ideally, all results should be successful capture like [this](#sExp).<br>
However, in reality, there are some conditions like [this](#fExp) that when a brand and some key attributes are mentioned together, but not correlated.<br>
These are the pros and cons about bag-of-words.